### **Attention is a technique that lets a model focus on the most relevant parts of the input when producing an output — instead of treating everything as equally important.**

- Think of it like how humans read:
When you read a sentence, you don’t remember every word equally — your brain pays more attention to the important words.

 Simple Example (Human-style):

Sentence: “The cat sat on the mat because it was tired.”

Question: What does “it” refer to?

Your brain instantly links “it” → “cat”, not “mat”.

### **Types of attention mechanisms:-** 
1. Simplified Self-Attention (Each word decides which other words are important)

2. Self-Attention (Each word uses query, key, and value to understand context from all words)

3. Causal Attention (Each word can attend only to past words, not future ones)

4. Multi-Head Attention (Multiple attention heads look at the same sentence from different perspectives).

<div class= "alert alert-block">

### **1. Simplified Self-Attention means:**

- Each word looks at other words in the same sentence

- It decides which words are important for understanding meaning

- Every word builds its meaning using context from related words

*Example:*

“I went to the bank to deposit money”

The word “bank” checks all other words

It gives high attention to:  “deposit”, “money”

It gives low attention to:  “river” (not present, but possible meaning)

- So the model understands bank = financial bank, not river bank
</div>

In [24]:
import torch

inputs = torch.tensor([
    [0.43, 0.15, 0.89], # your
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55] # step
])

In [25]:
query = inputs[1]

attention_score = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attention_score[i] = torch.dot(x_i, query)

print(attention_score)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [26]:
attention_weights = attention_score/attention_score.sum()
print("Attention weights : ", attention_weights)
print("Sum : ", attention_weights.sum())

Attention weights :  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum :  tensor(1.0000)


In [27]:
attention_weights_2 = torch.softmax(attention_score, dim=0)
print("Attention weights : ", attention_weights_2)
print("Sum : ", attention_weights_2.sum())

Attention weights :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [28]:
attention_scores = torch.zeros(6, 6)
for i, x_i in enumerate(inputs):        # using two for loops are generally slow
    for j, x_j in enumerate(inputs):
        attention_scores[i, j] = torch.dot(x_i, x_j)

attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

#### **instead of all of this, we can simply do the matrix multiplication of input with transpose of input**

In [29]:
attention_scores =  inputs @ inputs.T   # matrix multiplication of input matrix with transpose of input matrx
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [30]:
attention_weights = torch.softmax(attention_scores,dim= -1)
attention_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [31]:
print("All rows sum : ", attention_weights.sum(dim=-1))

All rows sum :  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [32]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

<div class = "alert alert-block">

### **2. Self-attention means:**
- Words attend to other words in the same sentence

- Every word understands its context

*Example:*

“I went to the bank to deposit money”

The word “bank” attends strongly to “deposit” and “money”, not “river”

In self-attention, every word is split into three roles:

1️⃣ **Query (Q)**

👉 What this word is looking for

Each word asks:
“Which other words are important for me?”

Query is used to search among other words.

*Think:* question in your head

2️⃣ **Key (K)**

👉 What each word represents

Every word advertises what it contains

Queries compare against keys to check relevance

*Think:* label or identity card

3️⃣ **Value (V)**

👉 The actual information to take

If a word is important, its value is used

Values are mixed to form the final meaning

*Think:* actual content
</div>

In [33]:
import torch

inputs = torch.tensor([
    [0.43, 0.15, 0.89], # your
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55] # step
])

In [34]:
x2 = inputs[1]      # second word (journey)
d_inp = inputs.shape[1]     # input embedding size i.e 3
d_out = 2       # output dimension size

In [35]:
torch.manual_seed(64)
W_query = torch.nn.Parameter(torch.rand(d_inp, d_out), requires_grad= False)
W_key = torch.nn.Parameter(torch.rand(d_inp, d_out), requires_grad= False)
W_value = torch.nn.Parameter(torch.rand(d_inp, d_out), requires_grad= False)
print(W_query)
print(W_key)
print(W_value)

Parameter containing:
tensor([[0.0492, 0.5989],
        [0.1771, 0.2639],
        [0.4717, 0.8384]])
Parameter containing:
tensor([[0.1639, 0.1899],
        [0.9971, 0.5516],
        [0.7827, 0.1247]])
Parameter containing:
tensor([[0.2144, 0.6680],
        [0.4840, 0.3438],
        [0.8777, 0.9955]])


In [36]:
queries = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value
query

tensor([0.5500, 0.8700, 0.6600])

In [37]:
attention_scores = queries @ keys.T
attention_scores

tensor([[0.7159, 1.3847, 1.3595, 0.8283, 0.5216, 1.1199],
        [0.7578, 1.4675, 1.4408, 0.8779, 0.5534, 1.1866],
        [0.7440, 1.4431, 1.4169, 0.8634, 0.5450, 1.1665],
        [0.4014, 0.7712, 0.7570, 0.4611, 0.2887, 0.6245],
        [0.2868, 0.5980, 0.5879, 0.3590, 0.2399, 0.4772],
        [0.5633, 1.0631, 1.0432, 0.6351, 0.3914, 0.8636]])

In [38]:
dim_key = inputs.shape[1]
attention_weights = torch.softmax(attention_scores / dim_key**0.5, dim=-1)
print(attention_weights)

tensor([[0.1400, 0.2059, 0.2029, 0.1493, 0.1251, 0.1767],
        [0.1383, 0.2083, 0.2051, 0.1482, 0.1229, 0.1771],
        [0.1387, 0.2076, 0.2045, 0.1486, 0.1236, 0.1770],
        [0.1521, 0.1883, 0.1867, 0.1574, 0.1425, 0.1730],
        [0.1534, 0.1836, 0.1825, 0.1599, 0.1493, 0.1712],
        [0.1473, 0.1965, 0.1943, 0.1535, 0.1333, 0.1751]])


In [39]:
context_vectors = attention_weights @ values
context_vectors

tensor([[0.8796, 1.0495],
        [0.8820, 1.0516],
        [0.8812, 1.0509],
        [0.8618, 1.0342],
        [0.8558, 1.0291],
        [0.8707, 1.0418]])

#### **now doing all the process in structure way by creating a class named SelfAttention_v1**

In [40]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_inp, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_inp, d_out))
        self.W_key = nn.Parameter(torch.rand(d_inp, d_out))
        self.W_value = nn.Parameter(torch.rand(d_inp, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attention_scores = queries @ keys.T
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5, dim= -1
        )

        context_vectors = attention_weights @ values
        return context_vectors

In [41]:
torch.manual_seed(64)
self_attent_v1 = SelfAttention_v1(d_inp, d_out)
print(self_attent_v1(inputs))

tensor([[0.8885, 1.0573],
        [0.8913, 1.0598],
        [0.8905, 1.0590],
        [0.8669, 1.0386],
        [0.8596, 1.0323],
        [0.8777, 1.0479]], grad_fn=<MmBackward0>)


#### **now using nn.Linear instead of nn.Parameter, contributing to more stable and effective model trainig**

In [42]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_inp, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_inp, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_inp, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_inp, d_out, bias = qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attention_scores = queries @ keys.T
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5, dim= -1
        )

        context_vectors = attention_weights @ values
        return context_vectors

In [43]:
torch.manual_seed(64)
self_attent_v2 = SelfAttention_v2(d_inp, d_out)
print(self_attent_v2(inputs))

tensor([[-0.0401,  0.4704],
        [-0.0400,  0.4726],
        [-0.0400,  0.4726],
        [-0.0393,  0.4764],
        [-0.0405,  0.4740],
        [-0.0389,  0.4763]], grad_fn=<MmBackward0>)


<div class = "alert alert-block">

### **3. Causal Self-Attention(Masked Attention) means:**

- Each word attends only to itself and previous words

- A word cannot see future words

- Used when generating text step-by-step

*Example:* “I am learning deep learning”

When predicting each word:

-  “I” → can see only “I”

-  “am” → can see “I am”
  
-  “learning” → can see “I am learning”
  
-  “deep” → can see previous words
  
-  “learning” → can see all previous words

*But:*

“I” cannot see “learning”

“am” cannot see “deep”

No word can look ahead
</div>

In [44]:
import torch

inputs = torch.tensor([
    [0.43, 0.15, 0.89], # your
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55] # step
])

queries = self_attent_v2(inputs)
keys = self_attent_v2(inputs)
attention_scores = queries @ keys.T
attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
attention_weights

tensor([[0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668]],
       grad_fn=<SoftmaxBackward0>)

In [51]:
context_length = attention_scores.shape[0]
mask_sample = torch.tril(torch.ones(context_length, context_length))
mask_sample

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [53]:
masked_sample = attention_weights * mask_sample
masked_sample

tensor([[0.1665, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1665, 0.1666, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1665, 0.1666, 0.1666, 0.0000, 0.0000, 0.0000],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.0000, 0.0000],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.0000],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668]],
       grad_fn=<MulBackward0>)

In [ ]:
row_sums = masked_sample.sum(dim=1, keepdim=True)
masked_sample_norm = masked_sample / row_sums
masked_sample_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4998, 0.5002, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3332, 0.3334, 0.3334, 0.0000, 0.0000, 0.0000],
        [0.2498, 0.2500, 0.2500, 0.2503, 0.0000, 0.0000],
        [0.1998, 0.2000, 0.2000, 0.2002, 0.2001, 0.0000],
        [0.1665, 0.1666, 0.1666, 0.1668, 0.1667, 0.1668]],
       grad_fn=<DivBackward0>)